# Filesystem

In this page considered tools that are used for system management in linux.

## Files and directories

There are tools that allow to manipulate with files and folders using Linux command system.

| Utility           | Description                                                 |
|-------------------|-------------------------------------------------------------|
| `ls`              | List files and directories.                                 |
| `cd`              | Change directory.                                           |
| `mkdir`           | Create directories.                                         |
| `rmdir`           | Remove empty directories.                                   |
| `rm`              | Remove files and directories.                               |
| `mv`              | Move or rename files and directories.                      |
| `cp`              | Copy files and directories.                                 |
| `touch`           | Create empty files or update timestamps.                   |
| `find`            | Search for files and directories.                          |
| `locate`          | Quickly find files by name (uses a prebuilt database).      |
| `tree`            | Display directories as a tree structure.                   |

Check details on the [specific page](filesystem/files_directories.ipynb).

## Disk usage (du)

The `du` command is used to check disk usage by different paths in the filesystem. It provides information about how much space is being used by files and directories.

---

The following cell creates several folders and files. Notably, `linux/du_example/megabytes_file` is created with a size of exactly 2.5 megabytes, whereas `linux/du_example/folder/small_file` contains only a single short line, making it an extremely small file.

In [ ]:
mkdir linux/du_example
mkdir linux/du_example/folder

dd if=/dev/zero of=linux/du_example/megabutes_file bs=1M count=2 &> /dev/null
dd if=/dev/zero of=linux/du_example/megabutes_file bs=512K count=1 oflag=append conv=notrunc &>/dev/null

echo "this is short message" >> linux/du_example/folder/small_file

Now let's try the `du` command. The following options are also added:

- `a`: prints both files and folders.
- `h`: displays file sizes in a human-readable format.

These options are really useful in my opinion.

In [ ]:
du -ah linux/du_example/

2,5M	linux/du_example/megabutes_file
4,0K	linux/du_example/folder/small_file
8,0K	linux/du_example/folder
2,6M	linux/du_example/


After all don't forget foder that was used for experiments.

In [ ]:
rm -r linux/du_example

## Archiving

This section discusses utilities in Linux related to archiving: combining a set of files into a single compact file and extracting it back to the original files. Below is a list of popular archive utilities:


| Utility  | Description                                                                                 |
|----------|---------------------------------------------------------------------------------------------|
| `tar`    | A widely-used tool for creating, extracting, and managing tarball archives (e.g., `.tar`, `.tar.gz`). |
| `gzip`   | Compresses files using the GNU zip algorithm, typically creating `.gz` files.               |
| `bzip2`  | Compresses files using the Burrows-Wheeler algorithm, typically creating `.bz2` files.      |
| `xz`     | Compresses files with high compression efficiency, typically creating `.xz` files.          |
| `zip`    | Creates compressed archives in `.zip` format, commonly used for cross-platform compatibility. |
| `unzip`  | Extracts `.zip` files.                                                                      |
| `7z`     | A high-compression utility for `.7z` format and other archive types, part of the p7zip package. |
| `ar`     | Creates, modifies, and extracts archives, often used for `.deb` packages in Debian-based systems. |
| `rar`    | Creates RAR archives, known for good compression ratios; proprietary software.               |
| `unrar`  | Extracts RAR files.                                                                         |
| `lzma`   | Compresses files using LZMA (Lempel-Ziv-Markov chain algorithm), predecessor to `xz`.       |
| `tar` + `lzma` | Combines tar archiving and LZMA compression, resulting in `.tar.lzma` files.          |
| `tar` + `xz`   | Combines tar archiving and XZ compression, resulting in `.tar.xz` files.              |
| `zstd`   | Compresses files with high speed and efficiency, creating `.zst` files.                     |
| `cpio`   | Archives files for use with tape backups or streams.                                        |

Find out more in the [particular page](filesystem/archiving.ipynb).

---

Consider the example of an archive file generated in the following cell.

In [ ]:
for ((i=0; i<1000000; i++))
do
    echo -n "a" >> /tmp/archive_me
done

du -h /tmp/archive_me

980K	/tmp/archive_me


File just contains repeated `a`, it is repeated so many times that all files take `980K`.

The next cell applies archiving to the created file.

In [ ]:
tar -cJvf /tmp/archive.tar.xz /tmp/archive_me
du -h /tmp/archive.tar.xz

tar: Removing leading `/' from member names
/tmp/archive_me
4,0K	/tmp/archive.tar.xz


Result takes only `4KB`.

The following cell restores the original file.

In [ ]:
mkdir /tmp/unarchived
tar -xJvf /tmp/archive.tar.xz -C /tmp/unarchived

tmp/archive_me


This is the file tree we got after unarchiving.

In [ ]:
tree /tmp/unarchived

/tmp/unarchived
└── tmp
    └── archive_me

1 directory, 1 file


Finally, check that the contents of the file have been restored correctly.

In [ ]:
head -c 100 /tmp/unarchived/tmp/archive_me

aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


## Permission/ownership 

Each file in Linux contains special attributes that specify permissions and ownership for that file. Permissions determine the ability of a user or group to read, write, or execute a file.

There are central commands to handle permissions and ownership:

- `ls -l`: Shows the content of a folder, along with information about the owner and permissions for each item.
- `chown`: Allows you to change the owner of a file or folder.
- `chmod`: Allows you to change the permissions of a file.

---

The following cell creates a file and shows its initial attributes:

In [46]:
[ -d /tmp/permissions ] && rm -r /tmp/permissions
mkdir /tmp/permissions
cd /tmp/permissions
touch toy_file
ls -l

total 0
-rw-r--r-- 1 root root 0 Feb  2 18:14 toy_file


Here, `-rw-r--r--` corresponds to the permissions, and `root root` describes the user and group, respectively.

The following cell creates a new user and grants them permissions for that file using the `chown` command.

In [42]:
useradd perm_exam
chown -c perm_exam toy_file 
ls -l
userdel perm_exam

changed ownership of 'toy_file' from root to perm_exam
total 0
-rw-r--r-- 1 perm_exam root 0 Feb  2 18:08 toy_file


As a result, in the output of `ls -l`, `perm_exam` is listed in the user field.

Finally, the last cell changes the permission attributes of the file using the `chmod` command.

In [45]:
chmod 417 toy_file
ls -l

total 0
-r----xrwx 1 1001 root 0 Feb  2 18:08 toy_file


It changes the first field corresponding to the `toy_file` - so the permissions are now different.

## `test`

`test` is a core linux utility that allows you to build expressions that depend on files. For more details see the [corresponding page](https://www.gnu.org/software/coreutils/manual/html_node/test-invocation.html) in the GNU manual. 

**Note:** for this command there is a very popular alias `[ <parameters> ] `.

---

Consider the basic example of using `test`. The following cell tries to check if the file `/tmp/wow` exists.

In [11]:
test -f /tmp/wow
echo $?

1


Since it wasn't created, we got the typical error result output - 1.

The following cell repeates previous experiment but now, it creates file before running `test -f /tmp/wow`.

In [13]:
echo "info" > /tmp/wow
test -f /tmp/wow
echo $?
rm /tmp/wow

0


So the result code is `0`, that means that file we checked truly exists.

### Negation

You can negotiate the test by using `!` in the list of parameters. 

---

For example, `! -f` will return true if there is no file in the given path. The following cell just shows how it works for a file that doesn't exist - there's nothing in the result, which means the cell exits with a 0 status code.

In [2]:
test ! -f /tmp/wow

This cell creates a file before executing `test !` as the result there is a `1` status code.

In [ ]:
echo "info" > /tmp/wow
test ! -f /tmp/wow
echo $?
rm /tmp/wow

1
